# Enums and Patterns
## Enums
By default, Rust stores C-style enums using the smallest built-in integer type that can accommodate them. Most fit in a single byte:

In [2]:
enum Ordering { 
    Less,
    Equal,
    Greater, 
}

enum HttpStatus { 
    Ok = 200,
    NotModified = 304,
    NotFound = 404,
}

use std::mem::size_of;
assert_eq!(size_of::<Ordering>(), 1); 
assert_eq!(size_of::<HttpStatus>(), 2); // 404 doesn't fit in a u8

Casting a C-style enum to an integer is allowed:

In [3]:
assert_eq!(HttpStatus::Ok as i32, 200);

However, casting in the other direction, from the integer to the enum, is not. Unlike C and C++, Rust guarantees that an enum value is only ever one of the values spelled out in the enum declaration. An unchecked cast from an integer type to an enum type could break this guarantee, so it’s not allowed. You can either write your own checked conversion:

In [4]:
fn http_status_from_u32(n: u32) -> Option<HttpStatus> { 
    match n {
        200 => Some(HttpStatus::Ok),
        304 => Some(HttpStatus::NotModified),
        404 => Some(HttpStatus::NotFound),
        _ => None, 
    }
}

or use the `enum_primitive crate`. It contains a macro that autogenerates this kind of conversion code for you.

As with structs, the compiler will implement features like the `==` operator for you, but you have to ask:


In [5]:
#[derive(Copy, Clone, Debug, PartialEq, Eq)]
enum TimeUnit {
    Seconds, Minutes, Hours, Days, Months, Years,
}

Enums can have methods, just like structs:


In [6]:
impl TimeUnit {
    /// Return the plural noun for this time unit. 
    fn plural(self) -> &'static str {
        match self {
            TimeUnit::Seconds => "seconds", 
            TimeUnit::Minutes => "minutes", 
            TimeUnit::Hours => "hours", 
            TimeUnit::Days => "days", 
            TimeUnit::Months => "months", 
            TimeUnit::Years => "years",
        } 
    }
        /// Return the singular noun for this time unit.
    fn singular(self) -> &'static str { 
        self.plural().trim_end_matches('s')
    } 
}

### Enums with Data
Some programs always need to display full dates and times down to the millisecond, but for most applications, it’s more user-friendly to use a rough approximation, like “two months ago.” We can write an enum to help with that, using the enum defined earlier:

In [8]:
/// A timestamp that has been deliberately rounded off, so our program 
/// says "6 months ago" instead of "February 9, 2016, at 9:49 AM". 
#[derive(Copy, Clone, Debug, PartialEq)]
enum RoughTime { 
    InThePast(TimeUnit, u32), JustNow, InTheFuture(TimeUnit, u32),
}

let four_score_and_seven_years_ago = 
    RoughTime::InThePast(TimeUnit::Years, 4 * 20 + 7);
let three_hours_from_now = 
    RoughTime::InTheFuture(TimeUnit::Hours, 3);

Enums can also have `struct variants`, which contain named fields, just like ordinary structs:

```Rust
enum Shape {
    Sphere { center: Point3d, radius: f32 }, 
    Cuboid { corner1: Point3d, corner2: Point3d },
}
let unit_sphere = Shape::Sphere { 
    center: ORIGIN,
    radius: 1.0,
};
```

In all, Rust has three kinds of enum variant, echoing the three kinds of struct we showed in the previous chapter. Variants with no data correspond to unit-like structs. Tuple variants look and function just like tuple structs. Struct variants have curly braces and named fields. A single enum can have variants of all three kinds:
```Rust
enum RelationshipStatus { 
    Single,
    InARelationship,
    ItsComplicated(Option<String>),
    ItsExtremelyComplicated {
        car: DifferentialEquation,
        cdr: EarlyModernistPoem, 
    },
}
```
**All constructors and fields of an enum share the same visibility as the enum itself.**

### Enums in Memory
In memory, enums with data are stored as a small integer tag, plus enough memory to hold all the fields of the largest variant. The tag field is for Rust’s internal use. It tells which constructor created the value and therefore which fields it has.

![image.png](asset/ch10/1.png)

Rust makes no promises about enum layout, however, in order to leave the door open for future optimizations. In some cases, it would be possible to pack an enum more efficiently than the figure suggests. For instance, some generic structs can be stored without a tag at all, as we’ll see later.

### Rich Data Structures Using Enums
Enums are also useful for quickly implementing tree-like data structures. For exam‐ ple, suppose a Rust program needs to work with arbitrary JSON data. In memory, any JSON document can be represented as a value of this Rust type:
```Rust
use std::collections::HashMap;
enum Json { 
    Null,
    Boolean(bool),
    Number(f64),
    String(String),
    Array(Vec<Json>), 
    Object(Box<HashMap<String, Json>>),
}
```
The explanation of this data structure in English can’t improve much upon the Rust code. The JSON standard specifies the various data types that can appear in a JSON document: null, Boolean values, numbers, strings, arrays of JSON values, and objects with string keys and JSON values. The Json enum simply spells out these types.

![image.png](asset/ch10/2.png)

### Generic Enums
Enums can be generic. Two examples from the standard library are among the most-used data types in the language:
```Rust
enum Option<T> { 
    None,
    Some(T), 
}

enum Result<T, E> { 
    Ok(T),
    Err(E), 
}
```

In [28]:
// An ordered collection of `T`s.
enum BinaryTree<T> { 
    Empty,
    NonEmpty(Box<TreeNode<T>>),
}
// A part of a BinaryTree.
struct TreeNode<T> { 
    element: T,
    left: BinaryTree<T>,
    right: BinaryTree<T>, 
}

let node: Box<TreeNode<i32>> = Box::new(
    TreeNode{
        element: 100, 
        left: BinaryTree::Empty, 
        right: BinaryTree::Empty
    }
);
let root: BinaryTree<i32> = BinaryTree::NonEmpty(node);

use self::BinaryTree::*;
let jupiter_tree = NonEmpty(
    Box::new(
        TreeNode {
            element: "Jupiter", left: Empty, right: Empty,
        }
    )
);

These few lines of code define a `BinaryTree` type that can store any number of values of type `T`.

Now comes the “price” we mentioned in the introduction. The tag field of an enum costs a little memory, up to eight bytes in the worst case, but that is usually negligible. The real downside to enums (if it can be called that) is that Rust code cannot throw caution to the wind and try to access fields regardless of whether they are actually present in the value:
```Rust
let r = shape.radius; // error: no field `radius` on type `Shape` 
```
The only way to access the data in an enum is the safe way: using patterns.

## Patterns